In [ ]:
%env LC_ALL=C
from cgitb import text
import spacy
import csv
import pandas as pd
import numpy as np
from tqdm import tqdm
from spacy.tokens import DocBin
from multiprocessing import Process, freeze_support, set_start_method

In [ ]:
def load_word_vectors(model_name, word_vectors):
    import spacy
    import subprocess
    import sys
    subprocess.run([sys.executable,
                    "-m",
                    "spacy",
                    "init-model",
                    "en",
                    model_name,
                    "--vectors-loc",
                    word_vectors
                        ]
                    )
    print (f"New spaCy model created with word vectors. File: {model_name}")

In [ ]:

if __name__ == '__main__':
    spacy.require_gpu()
    load_word_vectors("./models/model-best", "dataset/word_vecs.txt")
    nlp = spacy.load("./models/model-best")

    # for ent in doc.ents:
    #     print (ent.text, ent.label_)

    dataframe = pd.read_csv('./dataset/Listing_Titles.tsv', sep='\t', on_bad_lines='skip', quoting=csv.QUOTE_NONE, encoding='utf8')

    outsize = 1024 * 1024 * 1024 # 1GB

In [ ]:
    listing_titles = [(dataframe["Title"][i], i) for i in range(0, len(dataframe["Title"]))][5000:30000]

In [ ]:
    result = []

    with tqdm(total = 25000) as pbar:
        buffer = ""
        for doc, i in nlp.pipe(listing_titles, as_tuples=True, batch_size=10000):
            for ent in doc.ents:
                if(ent.label_ != "No Tag" and ent.label_ != "Obscure"):
                    result.append(str(i + 1) + '\t' + ent.label_ + '\t' + ent.text)
            pbar.update(1)
            
    output = open("output.tsv", "w", encoding="utf-8")  # write mode
    output.write('\n'.join(result))
    output.close()